In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
if tf.__version__.startswith("2."):
    import tensorflow.compat.v1 as tf
    tf.compat.v1.disable_v2_behavior()
    tf.compat.v1.disable_eager_execution()
    print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [3]:
dataSet = np.load('proyecto_training_data.npy')

In [4]:
dataP = dataSet[:int(len(dataSet)*0.8)]
dataA = dataSet[int(len(dataSet)*0.8):]
print(len(dataSet))
print(len(dataP))
print(len(dataA))
df = pd.DataFrame(dataP, columns = ['X0','X1','X2','X3','X4','X5'])
dfA = pd.DataFrame(dataA, columns = ['X0','X1','X2','X3','X4','X5'])
#print(df)
#dfA.describe()

1460
1168
292


In [5]:
#VALOR DE Y
Y = df['X0'].values
#entrenado X1 
X1 = df['X1'].values
#funcionLineal(X1,Y,10000,950,0.01,0,0,'X1')


In [ ]:
import 

In [6]:
def regresion_lineal(X_1,Y_1,epochs,lr,pol_n, mini_batch):
    
    tf.reset_default_graph()
    g = tf.Graph()
    
    with g.as_default():
        #**ejemplo de internet
        # x and y are placeholders for our training data
        x = tf.placeholder("float", name="X")
        y = tf.placeholder("float", name="Y")
        l_r = tf.placeholder("float", name ="lr")


        m = tf.Variable(0.0, name="m")
        b = tf.Variable(0.0, name="b")
        # Our model of y = a*x + b
        with tf.name_scope("modelo"):
            y_hat = tf.add(tf.multiply(x, m, name ="m.x"), b, name="b") #y = mx + b

        # Our error is defined as the square of the differences
        with tf.name_scope("error"):
            error = tf.math.pow((y - y_hat),2)/2
            error_promedio = tf.math.reduce_mean(error)

        #gradiente M
        with tf.name_scope("gradienteM"):
            gradiente_m = tf.multiply((y_hat - y),x)
            mean_grad_m = tf.math.reduce_mean(gradiente_m)

        #gradiente B
        with tf.name_scope("gradienteB"):
            gradiente_b = (y_hat - y) * 1
            mean_grad_b = tf.math.reduce_mean(gradiente_b)#valor promedio del gradiente de B

        error_summary = tf.summary.scalar(name ="error_promedio", tensor=error_promedio)

        #nuevo valor de la pendiente M
        nueva_m =  tf.assign(m, m - tf.multiply(l_r , mean_grad_m))#m

        #nuevo valor del intercepto B 
        nueva_b =  tf.assign(b, b - tf.multiply(l_r , mean_grad_b))#

        # Normal TensorFlow - initialize values, create a session and run the model
        model = tf.global_variables_initializer()

    with tf.Session(graph=g) as session:
        #
        writer = tf.summary.FileWriter('./graphs/lr='+str(lr)+'polinomio={0}'.format(pol_n), session.graph)
        
        session.run(model)#inicializador de variables
        total_iteraciones = int(len(Y_1)/mini_batch)
        
        point = 0
        
        for epoch in range(epochs):
            
            #np.random.shuffle(X_1,Y_1)
            
            #dict_f = {x: X_1, y:Y_1, l_r: lr}
            
            for i in range(total_iteraciones):
                #aplicando mini-batch
                muestra_inicio = i*mini_batch
                muestra_fin = muestra_inicio + mini_batch
                x_mb =  np.array(X_1[muestra_inicio:muestra_fin]).reshape([-1,1])
                y_mb = np.array(Y_1[muestra_inicio:muestra_fin]).reshape([-1,1])
                
                #feed_dict = {tensor_x:x_mb, tensor_y:y_mb}
                dict_f = {x: x_mb, y:y_mb, l_r: lr}
                #err,summary = session.run([error_promedio,error_summary], feed_dict=dict_f)
                n_m,n_b = session.run([nueva_m,nueva_b], feed_dict=dict_f)
                point+=1
                writer.add_summary(summary, point)
                #print(err,summary)
            err,summary = session.run([error_promedio,error_summary], feed_dict=dict_f)
        
        print("NUmero de iteraciones: {0}".format(point))
            
    #w_value = session.run(w)
    #print("Predicted model: {a:.3f}x + {b:.3f}".format(a=w_value[0], b=w_value[1]))

In [16]:
regresion_lineal(X1,Y,350,0.001,"j",32)

NUmero de iteraciones: 12600


In [8]:
len(X1)

1168

In [11]:
logs_base_dir = "./graphs"
#os.makedirs(logs_base_dir, exist_ok=True)
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
%tensorboard --logdir './graphs/' --port 6007

Reusing TensorBoard on port 6007 (pid 7392), started 5:07:47 ago. (Use '!kill 7392' to kill it.)

### **Experimento 1 probando un mini_batch de 2, con lr=0.001, y 50 iteraciones**

In [19]:
### **Experimento 1 probando un mini_batch de 2, con lr=0.001, y 50 iteraciones**
regresion_lineal(X1,Y,50,0.001,"a",2)

NUmero de iteraciones: 29200


In [29]:
### **Experimento 2 probando un mini_batch de 4, con lr=0.001, y 100 iteraciones**
regresion_lineal(X1,Y,100,0.001,"b",4)

NUmero de iteraciones: 29200


In [21]:
### **Experimento 3 probando un mini_batch de 8, con lr=0.001, y 200 iteraciones**
regresion_lineal(X1,Y,200,0.001,"c",8)

NUmero de iteraciones: 29200


In [22]:
### **Experimento 4 probando un mini_batch de 16, con lr=0.001, y 400 iteraciones**
regresion_lineal(X1,Y,400,0.001,"d",16)

NUmero de iteraciones: 29200


In [23]:
### **Experimento 5 probando un mini_batch de 32, con lr=0.001, y 800 iteraciones**
regresion_lineal(X1,Y,800,0.001,"e",32)

NUmero de iteraciones: 28800


In [24]:
### **Experimento 6 probando un mini_batch de 2, con lr=0.01, y 50 iteraciones**
regresion_lineal(X1,Y,50,0.01,"f",2)

NUmero de iteraciones: 29200


In [25]:
### **Experimento 7 probando un mini_batch de 4, con lr=0.01, y 100 iteraciones**
regresion_lineal(X1,Y,100,0.01,"g",4)

NUmero de iteraciones: 29200


In [26]:
### **Experimento 8 probando un mini_batch de 8, con lr=0.01, y 200 iteraciones**
regresion_lineal(X1,Y,200,0.01,"h",8)

NUmero de iteraciones: 29200


In [27]:
### **Experimento 9 probando un mini_batch de 16, con lr=0.01, y 400 iteraciones**
regresion_lineal(X1,Y,400,0.01,"i",16)

NUmero de iteraciones: 29200


In [28]:
### **Experimento 10 probando un mini_batch de 32, con lr=0.01, y 800 iteraciones**
regresion_lineal(X1,Y,800,0.01,"j",32)

NUmero de iteraciones: 28800


utilizar un enfoque investigación científica: antes del experimento formular una hipótesis del comportamiento experado, 
ejecutar el experimento y utilizar los resultados de tensorboard para conculir.

Dado que la variable utilizada tiene con coeficiente de correlacion de 0.79, se espera encontrar un modelo que minimice el error, sobre la cual se puedan hacer predicciones con un sesgo bajo.

In [ ]:
basado en los resultados experimentales obtenidos con tensorboard crear una conclusión general y elegir el mejor experimento.
- agregar el contenido de tensorboard al notebook(pueden ser screenshots=

Utilizando tensorboard como visualizador de MSE, se realizo una comparativa entre los modelos similares, es decir lr =0.001 con minibatch de 2, con el modelo con lr=0.01 y mismo minibatch, asi con todos los modelos experimentados.

Se puede observar que con el minibatch de 2, existe gran varianza del error en cuanto al modelo utilizado con batch gradient descent, pero que a medida que avanzan las iteraciones este modelo, el error va disminuyendo notoriamente, donde al final el sesgo visualizado se asemeja a la tendencia esperada, pero presenta mejores resultados en cuanto a sesgo el modelo con lr = 0.001.

En los modelos con minibatch de 8, se presenta un comportamiento interesante, donde se puede visualizar que ambos modelos tienen la misma tendencia en cuanto a la disminución del error, pero en este caso el modelo con lr=0.01 presenta mejores resultados, dado que el sesgo se apega mas a la tendencia esperada, pero ambos modelos terminan en un punto muy similar, dadas las iteraciones realizadas.

Pra el experimiento con minibatch de 32, y comparando ambos lr, se puede apreciar que el modelo que mejor representa el resultado esperado es con lr=0.01, donde la varianza del error es menor que con el modelo con lr=0.01.

Habiendo analizado todos los experimentos realizados, se concluye, que el mejor modelo para validar la hipotesis es el que utliza un lr=0.001 y mini-batch de 16, 




<img src="https://www.google.com.gt/url?sa=i&url=https%3A%2F%2Fwww.universoformulas.com%2Fmatematicas%2Fanalisis%2Ffuncion-exponencial%2F&psig=AOvVaw3Np-UMeO8iDj4kbSp945gq&ust=1623110052570000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCIir_KKahPECFQAAAAAdAAAAABAD">